Install various dependencies if not already installed

# Gathering Data


## Initialize

In [0]:
!pip install praw
!pip install twitterscraper
!pip install langdetect

     |████████████████████████████████| 143kB 2.7MB/s 
     |████████████████████████████████| 204kB 58.2MB/s 
     |████████████████████████████████| 92kB 5.0MB/s 
  Created wheel for twitterscraper: filename=twitterscraper-1.4.0-cp36-none-any.whl size=11347 sha256=1fbbb47b2618286e6e02de9d963617532976f7df5885b30a7b167c2231ea14b8
  Stored in directory: /root/.cache/pip/wheels/c2/9c/8b/7393e7bdc8abe6ce0d46c2ffae2035a1a2080a97ff0ddbdde6
Successfully built twitterscraper
     |████████████████████████████████| 1.0MB 2.5MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993460 sha256=5298d9d9ef3048fb5a179d58669d5595cf01c3e870d7e176b6ec76ca599e01af
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
import os
import xml, csv, json
import xml.etree.ElementTree as et
import praw
import pandas as pd
import datetime as dt
import re

from langdetect import detect
from twitterscraper import query_tweets

Lets setup some identity
- Google Drive
- Reddit user

In [0]:
from google.colab import drive
drive.mount('/content/drive')

reddit = praw.Reddit(client_id='reCMqTD0IKajPA', \
                     client_secret='aV4fLKBfmB_Nr_GZ6GQ8-neUfo8', \
                     user_agent='G09', \
                     username='teapotisquitesexy', \
                     password='g09aintgotnosecret')

if reddit.user.me():
  print("Reddit Credential verified.")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Reddit Credential verified.


## Question and statement Data

In [0]:
def cleanData():
  cleanQuestionData()
  cleanStatementData()
  

def cleanQuestionData():
  files = [
    '/content/drive/My Drive/G09 Data/sports.xml',
    '/content/drive/My Drive/G09 Data/parenting.xml',
    '/content/drive/My Drive/G09 Data/christianity.xml',
    '/content/drive/My Drive/G09 Data/cooking.xml'
  ]

  roots = []
  for file in files:
    roots.append(et.parse(file).getroot())

  count = 0
  with open("/content/drive/My Drive/G09 Data/questions-cleaned.txt", "w") as cleanFile:

    for root in roots:
      for post in root.findall('row'):
        title = post.get('Title')
        if title:
          count +=1
          cleanFile.write(title + os.linesep)

    print("Extracted : "+str(count))

def cleanStatementData():
  with open('/content/drive/My Drive/G09 Data/News_Category_Dataset_v2.json') as fp:

    with open("/content/drive/My Drive/G09 Data/statement-cleaned.txt", "w") as cleanFile:
      line = fp.readline()
      cnt = 1
      while line:
        data = json.loads(line.strip())
        cleanFile.write(data.get('headline') + os.linesep)
        cleanFile.write(data.get('short_description') + os.linesep)
        

        line = fp.readline()
        cnt += 1
  
    print("Extracted : "+str(cnt))

# cleanQuestionData()
# cleanStatementData()

def previewTXT(filepath):

  preview_depth = 20
  with open(filepath) as txtFile:
    for line in range(preview_depth):
      print(txtFile.readline())

# previewTXT('/content/drive/My Drive/G09 Data/questions-cleaned.txt')


## Social Media Data


In [0]:
def removeInlineURL(s):
  s = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', s, flags=re.MULTILINE)
  return s

In [0]:
def crawReddit():

  post_dict = { 
      "source": [],
      "title": [],
      "score": [],
      "id": [], 
      "url": [],
      "comms_num": [],
      "created": [],
      "body": []
  }

  subreddit = reddit.subreddit('Vodafone')
  top_subreddit = subreddit.top()
  for submission in subreddit.top(limit=1010):

    # Clean text
    submission.title = removeInlineURL(submission.title)
    submission.selftext = removeInlineURL(submission.selftext)
    

    post_dict["source"].append("Reddit")
    post_dict["title"].append(submission.title)
    post_dict["score"].append(submission.score)
    post_dict["id"].append(submission.id)
    post_dict["url"].append(submission.url)
    post_dict["comms_num"].append(submission.num_comments)
    post_dict["created"].append(submission.created)
    post_dict["body"].append(submission.selftext)

  post_data = pd.DataFrame(post_dict)
  print(post_data)
  post_data.to_csv('/content/drive/My Drive/G09 Data/reddit-cleaned.csv')

crawReddit()

     source  ...                                               body
0    Reddit  ...  Has anyone tried the Gigacube and had any luck...
1    Reddit  ...  Just a heads up. I was a Spark customer paying...
2    Reddit  ...  how is it even possible that huge parts of the...
3    Reddit  ...                                                   
4    Reddit  ...                                                   
..      ...  ...                                                ...
213  Reddit  ...  No matter how many time I complain to them, th...
214  Reddit  ...  Can someone give me an honest review of using ...
215  Reddit  ...  Hi\n\nI'm on a UK Vodafone business contract f...
216  Reddit  ...  So I've been on a 20gb 4g unlim text unlim voi...
217  Reddit  ...                                                   

[218 rows x 8 columns]


In [0]:
twitterFeed = query_tweets("vodafone", 7000)

INFO: queries: ['vodafone since:2006-03-21 until:2006-11-29', 'vodafone since:2006-11-29 until:2007-08-09', 'vodafone since:2007-08-09 until:2008-04-18', 'vodafone since:2008-04-18 until:2008-12-27', 'vodafone since:2008-12-27 until:2009-09-06', 'vodafone since:2009-09-06 until:2010-05-17', 'vodafone since:2010-05-17 until:2011-01-26', 'vodafone since:2011-01-26 until:2011-10-06', 'vodafone since:2011-10-06 until:2012-06-15', 'vodafone since:2012-06-15 until:2013-02-23', 'vodafone since:2013-02-23 until:2013-11-03', 'vodafone since:2013-11-03 until:2014-07-14', 'vodafone since:2014-07-14 until:2015-03-24', 'vodafone since:2015-03-24 until:2015-12-03', 'vodafone since:2015-12-03 until:2016-08-12', 'vodafone since:2016-08-12 until:2017-04-22', 'vodafone since:2017-04-22 until:2017-12-31', 'vodafone since:2017-12-31 until:2018-09-10', 'vodafone since:2018-09-10 until:2019-05-21', 'vodafone since:2019-05-21 until:2020-01-30']
INFO: Querying vodafone since:2010-05-17 until:2011-01-26
INFO: 

In [0]:
def crawTwitter():

  tweet_dict = {
    "source": [],
    "id": [],
    "text":[],
    "timestamp":[]
  }
  
  for tweet in twitterFeed:
    try:
      # print(tweet.text)
      lang = detect(tweet.text)
      if lang == 'en':

        # Clean text
        tweet.text = removeInlineURL(tweet.text)
        
        tweet_dict["source"].append("Twitter")
        tweet_dict["id"].append(tweet.tweet_id)
        tweet_dict["text"].append(tweet.text)
        tweet_dict["timestamp"].append(tweet.timestamp)
    except Exception as e: print(e)

  tweet_data = pd.DataFrame(tweet_dict)
  print(tweet_data)
  tweet_data.to_csv('/content/drive/My Drive/G09 Data/twitter-cleaned.csv')

  return tweet_data
 
  
      

crawTwitter()

No features in text.
No features in text.
No features in text.
No features in text.
       source  ...           timestamp
0     Twitter  ... 2007-08-08 14:27:56
1     Twitter  ... 2007-08-08 13:20:14
2     Twitter  ... 2007-08-08 13:08:32
3     Twitter  ... 2007-08-08 11:37:18
4     Twitter  ... 2007-08-08 11:36:25
...       ...  ...                 ...
2419  Twitter  ... 2017-12-30 23:18:02
2420  Twitter  ... 2017-12-30 23:06:37
2421  Twitter  ... 2017-12-30 23:01:53
2422  Twitter  ... 2017-12-30 23:00:43
2423  Twitter  ... 2017-12-30 22:58:30

[2424 rows x 4 columns]


,source,id,text,timestamp
0,Twitter,194103532,@mikeypod i got that shit from vodafone. Some ...,2007-08-08 14:27:56
1,Twitter,193996122,Mobile phone giant Vodafone says it does not i...,2007-08-08 13:20:14
2,Twitter,193976372,Vodafone to keep Verizon stake: Read full stor...,2007-08-08 13:08:32
3,Twitter,193839232,Got my second Vodafone bill. Lots of roaming l...,2007-08-08 11:37:18
4,Twitter,193837962,Found and bought this cool 'VODAFONE Prepaid S...,2007-08-08 11:36:25
...,...,...,...,...
2419,Twitter,947245421646278657,"SAP HR Functional #consultant Lead: #Vodafone,...",2017-12-30 23:18:02
2420,Twitter,947242551786070019,My broadband number is 0302-427617 and my Voda...,2017-12-30 23:06:37
2421,Twitter,947241358917603328,#Vodafone pic.twitter.com/SPjrOr0A5C,2017-12-30 23:01:53
2422,Twitter,947241065249062912,Who just sent me Vodafone credits?,2017-12-30 23:00:43
